In [ ]:
# Python code in Google Colab
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 13.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661204 sha256=a8e9fb2a4c61ef74c07070cee32cee0e336a478d5fb9f6576b8b72efd653ae37
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [ ]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

In [ ]:
# Load CUDA kernel code from file
with open('reduction_kernel.cu', 'r') as file:
    reduction_kernel = file.read()


In [ ]:
# Compile the CUDA kernel
mod = SourceModule(reduction_kernel)

LogicError: cuModuleLoadDataEx failed: an illegal memory access was encountered - 

In [ ]:
# Get the functions from the compiled module
min_reduction = mod.get_function("min_reduction")
max_reduction = mod.get_function("max_reduction")
sum_reduction = mod.get_function("sum_reduction")
avg_reduction = mod.get_function("avg_reduction")


In [ ]:
# Example input data
input_data = np.array([0, 1, 2, 3, 4, 5], dtype=np.float32)  # Example input data (replace with your data)
data_size = input_data.size

# Allocate GPU memory for input and output data
input_gpu = cuda.mem_alloc(input_data.nbytes)
output_gpu = cuda.mem_alloc(input_data.nbytes)

# Copy input data to GPU memory
cuda.memcpy_htod(input_gpu, input_data)

# Define block and grid dimensions
block_dim = (256, 1, 1)
grid_dim = ((data_size - 1) // 256 + 1, 1)

# Execute the CUDA kernels for min, max, sum, and average
min_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)
max_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)
sum_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)
avg_reduction(input_gpu, output_gpu, np.int32(data_size), block=block_dim, grid=grid_dim)

# Copy the results back to the host
min_result = np.empty_like(input_data)
max_result = np.empty_like(input_data)
sum_result = np.empty_like(input_data)
avg_result = np.empty_like(input_data)

cuda.memcpy_dtoh(min_result, output_gpu)
cuda.memcpy_dtoh(max_result, output_gpu)
cuda.memcpy_dtoh(sum_result, output_gpu)
cuda.memcpy_dtoh(avg_result, output_gpu)

print("Input Data:", input_data)
print("Minimum Value:", min_result)
print("Maximum Value:", max_result)
print("Sum:", sum_result)
print("Average:", avg_result)

LogicError: cuMemcpyDtoH failed: an illegal memory access was encountered